In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Load database credentials from environment variables
db_user = os.getenv("PG_USER", "postgres")
db_pass = os.getenv("PG_PASSWORD", "isba_4715")
db_host = os.getenv("PG_HOST", "isba-dev-02.c8p8820o4mvz.us-east-1.rds.amazonaws.com")
db_port = os.getenv("PG_PORT", "5432")
db_name = os.getenv("PG_DB", "sql_project")

# Create the database connection
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

### Descriptive Analytics:
**Business Question: Which Apple stores in NYC have the highest Yelp ratings and how do they compare by location?**

In [2]:
query1 = '''
WITH avg_rating_by_city AS (
    SELECT "Address", "Rating",
           SPLIT_PART("Address", ',', 2) AS Borough
    FROM sql_project
),
ranked_stores AS (
    SELECT *,
           RANK() OVER (ORDER BY "Rating" DESC) AS rank_by_rating
    FROM avg_rating_by_city
)
SELECT *
FROM ranked_stores
ORDER BY rank_by_rating
LIMIT 10;
'''
df1 = pd.read_sql(query1, engine)
pd.set_option('display.max_rows', None)
df1

,Address,Rating,borough,rank_by_rating
0,"767 5th Ave, New York, NY 10153",3.4,New York,1
1,"103 Prince St, New York, NY 10012",3.3,New York,2
2,"401 W 14th St, New York, NY 10014",3.2,New York,3
3,"185 Greenwich St, New York, NY 10007",3.0,New York,4
4,"45 Grand Central Terminal, New York, NY 10017",2.9,New York,5
5,"1981 Broadway, New York, NY 10023",2.7,New York,6
6,"247 Bedford Ave, Brooklyn, NY 11211",2.6,Brooklyn,7
7,"123 Flatbush Ave, Brooklyn, NY 11217",2.6,Brooklyn,7
8,"940 Madison Ave, New York, NY 10021",2.5,New York,9
9,"90-15 Queens Blvd, Elmhurst, NY 11373",2.5,Elmhurst,9


**Insight**: Apple Stores in Manhattan consistently rank higher in customer ratings than those in others, revealing a concentration of positive experiences in central locations. The lowest-rated stores are in Brooklyn and Elmhurst.

**Recommendation**: Prioritize operational improvements in underperforming stores such as Brooklyn and Elmhurst by analyzing service gaps, staff training, or localized customer needs.

**Prediction**: With targeted improvements, Apple Stores in Brooklyn and Elmhurst have strong potential to close the rating gap and become top-performing locations in the NYC market.

### Diagnostic Analytics:
**Business Question: Which stores have the highest volume of reviews and do review counts correlate with ratings?**

In [2]:
query2 = '''
WITH review_stats AS (
    SELECT "Store Name", "Review Count", "Rating"
    FROM sql_project
),
avg_review AS (
    SELECT AVG("Review Count") AS avg_reviews, AVG("Rating") AS avg_rating FROM review_stats
)
SELECT r.*, a.avg_reviews, a.avg_rating
FROM review_stats r, avg_review a
ORDER BY r."Review Count" DESC
LIMIT 10;
'''
df2 = pd.read_sql(query2, engine)
df2

,Store Name,Review Count,Rating,avg_reviews,avg_rating
0,Apple Fifth Avenue,1194,3.4,429.1,2.87
1,Apple Soho,607,3.3,429.1,2.87
2,Apple Grand Central,566,2.9,429.1,2.87
3,Apple - New York,473,3.2,429.1,2.87
4,Apple - Upper West Side,463,2.7,429.1,2.87
5,Apple - Upper East Side,217,2.5,429.1,2.87
6,Apple - Williamsburg,206,2.6,429.1,2.87
7,Apple World Trade Center,202,3.0,429.1,2.87
8,Apple - Brooklyn,183,2.6,429.1,2.87
9,Apple Queens Center,180,2.5,429.1,2.87


**Insight**: Apple Fifth Avenue attracts significantly more reviews, indicating a strong link between foot traffic and online engagement.

**Recommendation**: Maximize marketing impact by prioritizing high-traffic locations and adapting proven strategies to boost visibility and engagement at underperforming stores.

**Prediction**: As low-performance stores gain exposure through strategic investment, both review volume and customer sentiment are likely to improve.